# Epoch

In [1]:
import numpy as np

In [2]:
class Tensor:

    def __init__(self, data):
        self.data = np.array(data)

        self.grad = None
        self.gradient_fn = lambda: None
        self.parents = set()

    def gradient(self):
        if self.gradient_fn:
            self.gradient_fn()

        for p in self.parents:
            p.gradient()

    def size(self):
        return self.data.shape[-1]

In [3]:
class DataLoader:

    def __init__(self, batch_size):
        self.batch_size = batch_size

        self.features = [[28.1, 58.0],
                         [22.5, 72.0],
                         [31.4, 45.0],
                         [19.8, 85.0],
                         [27.6, 63]]
        self.labels = [[165],
                       [95],
                       [210],
                       [70],
                       [155]]

    def __len__(self):  # 3
        return len(self.features)

    def __getitem__(self, index):  # 4
        return (Tensor(self.features[index: index + self.batch_size]),
                Tensor(self.labels[index: index + self.batch_size]))

In [4]:
class Linear:

    def __init__(self, in_size, out_size):
        self.in_size = in_size
        self.out_size = out_size

        self.weight = Tensor(np.ones((out_size, in_size)) / in_size)
        self.bias = Tensor(np.zeros(out_size))

    def __call__(self, x: Tensor):
        return self.forward(x)

    def forward(self, x: Tensor):
        p = Tensor(x.data @ self.weight.data.T + self.bias.data)

        def gradient_fn():
            self.weight.grad = p.grad.T @ x.data / len(x.data)
            self.bias.grad = np.sum(p.grad, axis=0) / len(x.data)

        p.gradient_fn = gradient_fn
        p.parents = {self.weight, self.bias}
        return p

    def parameters(self):
        return [self.weight, self.bias]

In [5]:
class MSELoss:

    def __call__(self, p: Tensor, y: Tensor):
        mse = Tensor(((p.data - y.data) ** 2).mean())

        def gradient_fn():
            p.grad = (p.data - y.data) * 2

        mse.gradient_fn = gradient_fn
        mse.parents = {p}
        return mse

In [6]:
class SGD:

    def __init__(self, parameters, lr):
        self.parameters = parameters
        self.lr = lr

    def backward(self):
        for p in self.parameters:
            p.data -= p.grad * self.lr

In [7]:
LEARNING_RATE = 0.00001
BATCHES = 2
EPOCHS = 100

In [8]:
dataset = DataLoader(BATCHES)

feature, label = dataset[0]
model = Linear(feature.size(), label.size())
loss = MSELoss()
sgd = SGD(model.parameters(), LEARNING_RATE)

for epoch in range(EPOCHS):
    print(f"epoch: {epoch}")

    for i in range(0, len(dataset), dataset.batch_size):
        feature, label = dataset[i]

        prediction = model(feature)
        print(f'prediction: {prediction.data}')

        error = loss(prediction, label)
        print(f'error: {error.data}')

        error.gradient()
        sgd.backward()
        print(f"weight: {model.weight.data}")
        print(f"bias: {model.bias.data}")

epoch: 0
prediction: [[43.05]
 [47.25]]
error: 8575.9325
weight: [[0.5450117 0.605111 ]]
bias: [0.001697]
prediction: [[44.34505938]
 [62.22736366]]
error: 13750.986613744806
weight: [[0.59856633 0.68626246]]
bias: [0.00343128]
prediction: [[59.75839732]]
error: 9070.962881280973
weight: [[0.6511397  0.80626688]]
bias: [0.00533611]
epoch: 1
prediction: [[65.06584087]
 [72.70719493]]
error: 5241.902659665337
weight: [[0.68423708 0.88027952]]
bias: [0.00655838]
prediction: [[61.10418082]
 [88.37821134]]
error: 11253.861810556275
weight: [[0.72735148 0.93166115]]
bias: [0.00786355]
prediction: [[78.77741711]]
error: 5809.882141806621
weight: [[0.76942635 1.02770161]]
bias: [0.00938801]
epoch: 2
prediction: [[81.23696162]
 [91.31599661]]
error: 3514.909239990455
weight: [[0.79379266 1.07893665]]
bias: [0.01026248]
prediction: [[ 73.4875014 ]
 [107.43697269]]
error: 10018.594599336657
weight: [[0.82924506 1.10854585]]
bias: [0.01125323]
prediction: [[92.73680561]]
error: 3876.7053755846787
